##**Crypto-Answer**
###Annika Köhler (2145451), Markus Schmidbauer (2144933) BA Medieninformatik
 
Cryptocurrencies as alternative investment forms are in high demand [1]. Cyptocurrency-Websites which display data including information about the price or the volume of each of the thousands currencies are confusing and overwhelming especially for newcomers [2][1]. A Question Answering (QA) system for cryptocurrency would fix this problem. The purpose of such a QA-System is to be simple and effective for a user to recieve an answer to a question on the topic of cryptocurrency.
In the following a Question-Answering System for Cryptocurrency is being implemented. 

We used Haystack to build our QA-system [3].

 
---
 
 
Make sure you open this notebook in a Chrome (or Chromium-based) browser, otherwise complication with the webcrawler could arise.



---

**The QA-systems architecture:**

<img height="400px" src= "https://drive.google.com/uc?export=view&id=1fy875F3MrEhpqgND7cfhA-FERmjBQCav">


##Installation

Make sure you enable the GPU runtime to experience decent speed.
 
Do the following steps:
 
 
 
1.   click in the menu-bar at the top on Runtime
2.   click in the drop-down menu on Change Runtime type
3.   for the Hardware accelerator select -> GPU
4.   click save

In [ ]:
#Make sure you enable the GPU runtime to experience decent speed
!nvidia-smi
 
#Installation of Haystack
!pip install farm-haystack
!pip install grpcio-tools
!pip install git+https://github.com/deepset-ai/haystack.git
 
#Install to be able to create a connection to our website
!pip install anvil-uplink

Fri Sep 10 11:25:01 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P8    35W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

     |████████████████████████████████| 62 kB 843 kB/s 
     |████████████████████████████████| 51 kB 194 kB/s 
  Created wheel for ws4py: filename=ws4py-0.5.1-py3-none-any.whl size=45230 sha256=cc9164e172b1c856f43d03f903655cae72e09b1eb9d11daa620cbff312287870
  Stored in directory: /root/.cache/pip/wheels/29/ea/7d/3410aa0aa0e4402ead9a7a97ab2214804887e0f5c2b76f0c96
Successfully built ws4py


These are necessary import-statements for the Pipeline

In [ ]:
import importlib
import pkg_resources
importlib.reload(pkg_resources)
import haystack
from haystack.reader.farm import FARMReader
from haystack.utils import print_answers
from haystack.retriever.sparse import ElasticsearchRetriever
import anvil.server

09/10/2021 11:29:00 - INFO - faiss.loader -   Loading faiss with AVX2 support.
09/10/2021 11:29:00 - INFO - faiss.loader -   Could not load library with AVX2 support due to:
ModuleNotFoundError("No module named 'faiss.swigfaiss_avx2'")
09/10/2021 11:29:00 - INFO - faiss.loader -   Loading faiss.
09/10/2021 11:29:00 - INFO - faiss.loader -   Successfully loaded faiss.
09/10/2021 11:29:00 - INFO - farm.modeling.prediction_head -   Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .


#Mount to GoogleDrive
 
We use google drive to mount our fine-tuning data file. 
 
The folder structure is as following:
 
  Colab Notebooks -> Crypto-Answer -> data -> squad20 -> training_data.json
 
 
---
 
To be able to use files from Google drive do the following steps:
 
 
 
1.   run this cell and click on the returned link
2.   select the google-account with which this notebook is connected
3.   click on log in
4.   copy the key
5.   enter the copied key into the displayed input field

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Architecture

##Reader and Fine Tuning

We used the pre-trained roberta-base-squad2 model from Huggingface as our base reader [4].

To fine-tune our reader we used Haystack's provided Annotation Tool [5], to train the reader on our own Question-Answer pairs, which are stored in the file "training_data_fine_tuning.json". 

-> Move your Folder either to the correct directory, or change data_dir to the location it is stored.


In [ ]:
#reader based on the pre-trained roberta-base-squad2 model from Huggingface
reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)

#directory for training_data_fine_tuning.json
data_dir = "/content/drive/MyDrive/Colab Notebooks/Crypto-Answer/data/squad20/"

#fine-tune the reader on our training_data_fine_tuning.json data from Haystack's Annotation Tool
reader.train(data_dir=data_dir, train_filename="training_data_fine_tuning.json", use_gpu=True, n_epochs=1, save_dir="my_model")

09/10/2021 11:30:53 - INFO - farm.utils -   Using device: CUDA 
09/10/2021 11:30:53 - INFO - farm.utils -   Number of GPUs: 1
09/10/2021 11:30:53 - INFO - farm.utils -   Distributed Training: False
09/10/2021 11:30:53 - INFO - farm.utils -   Automatic Mixed Precision: None
09/10/2021 11:30:53 - INFO - filelock -   Lock 140554614890640 acquired on /root/.cache/huggingface/transformers/c40d0abb589629c48763f271020d0b1f602f5208c432c0874d420491ed37e28b.122ed338b3591c07dba452777c59ff52330edb340d3d56d67aa9117ad9905673.lock


Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

09/10/2021 11:30:54 - INFO - filelock -   Lock 140554614890640 released on /root/.cache/huggingface/transformers/c40d0abb589629c48763f271020d0b1f602f5208c432c0874d420491ed37e28b.122ed338b3591c07dba452777c59ff52330edb340d3d56d67aa9117ad9905673.lock
09/10/2021 11:30:54 - INFO - filelock -   Lock 140554596357584 acquired on /root/.cache/huggingface/transformers/eac3273a8097dda671e3bea1db32c616e74f36a306c65b4858171c98d6db83e9.084aa7284f3a51fa1c8f0641aa04c47d366fbd18711f29d0a995693cfdbc9c9e.lock


Downloading:   0%|          | 0.00/496M [00:00<?, ?B/s]

09/10/2021 11:31:10 - INFO - filelock -   Lock 140554596357584 released on /root/.cache/huggingface/transformers/eac3273a8097dda671e3bea1db32c616e74f36a306c65b4858171c98d6db83e9.084aa7284f3a51fa1c8f0641aa04c47d366fbd18711f29d0a995693cfdbc9c9e.lock
Some weights of the model checkpoint at deepset/roberta-base-squad2 were not used when initializing RobertaModel: ['qa_outputs.bias', 'qa_outputs.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at deepset/roberta-base-squad2 and are newly initialized: ['ro

...

09/10/2021 11:31:20 - INFO - filelock -   Lock 140554599678608 released on /root/.cache/huggingface/transformers/b87d46371731376b11768b7839b1a5938a4f77d6bd2d9b683f167df0026af432.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b.lock
09/10/2021 11:31:21 - INFO - filelock -   Lock 140554600071824 acquired on /root/.cache/huggingface/transformers/c9d2c178fac8d40234baa1833a3b1903d393729bf93ea34da247c07db24900d0.cb2244924ab24d706b02fd7fcedaea4531566537687a539ebb94db511fd122a0.lock


Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

09/10/2021 11:31:22 - INFO - filelock -   Lock 140554600071824 released on /root/.cache/huggingface/transformers/c9d2c178fac8d40234baa1833a3b1903d393729bf93ea34da247c07db24900d0.cb2244924ab24d706b02fd7fcedaea4531566537687a539ebb94db511fd122a0.lock
09/10/2021 11:31:22 - INFO - filelock -   Lock 140554600073872 acquired on /root/.cache/huggingface/transformers/e8a600814b69e3ee74bb4a7398cc6fef9812475010f16a6c9f151b2c2772b089.451739a2f3b82c3375da0dfc6af295bedc4567373b171f514dd09a4cc4b31513.lock


Downloading:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

09/10/2021 11:31:22 - INFO - filelock -   Lock 140554600073872 released on /root/.cache/huggingface/transformers/e8a600814b69e3ee74bb4a7398cc6fef9812475010f16a6c9f151b2c2772b089.451739a2f3b82c3375da0dfc6af295bedc4567373b171f514dd09a4cc4b31513.lock
09/10/2021 11:31:23 - WARNING - farm.utils -   ML Logging is turned off. No parameters, metrics or artifacts will be logged to MLFlow.
09/10/2021 11:31:23 - INFO - farm.utils -   Using device: CUDA 
09/10/2021 11:31:23 - INFO - farm.utils -   Number of GPUs: 1
09/10/2021 11:31:23 - INFO - farm.utils -   Distributed Training: False
09/10/2021 11:31:23 - INFO - farm.utils -   Automatic Mixed Precision: None
09/10/2021 11:31:23 - INFO - farm.infer -   Got ya 2 parallel workers to do inference ...
09/10/2021 11:31:23 - INFO - farm.infer -    0    0 
09/10/2021 11:31:23 - INFO - farm.infer -   /w\  /w\
09/10/2021 11:31:23 - INFO - farm.infer -   /'\  / \
09/10/2021 11:31:23 - INFO - farm.infer -     
09/10/2021 11:31:23 - INFO - farm.utils -   Usi

##Document-store

Start Elasticsearch from source, to be able to use the Elasticsearch document-store [6].

This process requires a few seconds.


In [ ]:
! wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
! tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
! chown -R daemon:daemon elasticsearch-7.9.2

import os
from subprocess import Popen, PIPE, STDOUT
es_server = Popen(['elasticsearch-7.9.2/bin/elasticsearch'],
                   stdout=PIPE, stderr=STDOUT,
                   preexec_fn=lambda: os.setuid(1)
                  )

! sleep 30

We create an Elasticsearch document-stores for the retriever models.

In [ ]:
from haystack.document_store.elasticsearch import ElasticsearchDocumentStore
document_store = ElasticsearchDocumentStore(host="localhost", username="", password="", index="document")

09/10/2021 11:44:14 - INFO - elasticsearch -   HEAD http://localhost:9200/ [status:200 request:0.089s]
09/10/2021 11:44:15 - INFO - elasticsearch -   PUT http://localhost:9200/document [status:200 request:0.459s]
09/10/2021 11:44:15 - INFO - elasticsearch -   PUT http://localhost:9200/label [status:200 request:0.231s]
09/10/2021 11:44:15 - INFO - elasticsearch -   HEAD http://localhost:9200/ [status:200 request:0.008s]
09/10/2021 11:44:15 - INFO - elasticsearch -   HEAD http://localhost:9200/document [status:200 request:0.010s]
09/10/2021 11:44:15 - INFO - elasticsearch -   GET http://localhost:9200/document [status:200 request:0.007s]
09/10/2021 11:44:15 - INFO - elasticsearch -   PUT http://localhost:9200/document/_mapping [status:200 request:0.018s]
09/10/2021 11:44:15 - INFO - elasticsearch -   HEAD http://localhost:9200/label [status:200 request:0.004s]


This function saves the preprocessed dictionary into the elasticsearch document-store, after deleting already existing files. 

In [ ]:
def saveDataFromDictonaryToElasticseach(dicts):
    try:
      document_store.delete_documents()
    except:
      print("Document store is empty.")
    document_store.write_documents(dicts)

##Pre-processing


To implement pre-processing steps on the documents, we also used the code from Haystack's official github page [9]. 

The provided Pre-processor did not work on our customized dictonary, therefore we modified the most relevant functions.

These include cleaning unnecessary escape-sequences as well as splitting longer texts into multiple smaller paragraphs.

In [ ]:
import logging
import re
from copy import deepcopy
from functools import partial, reduce
from itertools import chain
from typing import List, Optional, Generator, Set, Union
from more_itertools import windowed
from tqdm import tqdm
from haystack.preprocessor.base import BasePreProcessor
logger = logging.getLogger(__name__)

def preprocessDicts(dicts):
  for dictionary in dicts:  
    text = dictionary["text"]
    lines = text.splitlines()

    cleaned_lines = []
    for line in lines:
      line = line.strip()
      line.replace("\n", " \n")
      line.replace("\t", " \t")
      cleaned_lines.append(line)
      text = "\n".join(cleaned_lines)
      text = re.sub(r"\n\n+", "\n\n", text)
      
    dictionary["text"] = text
  return dicts



In [ ]:
import logging
import re
from copy import deepcopy
from functools import partial, reduce
from itertools import chain
from typing import List, Optional, Generator, Set, Union
from more_itertools import windowed
from tqdm import tqdm
from haystack.preprocessor.base import BasePreProcessor
logger = logging.getLogger(__name__)
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize,sent_tokenize


def splitDictionaries(cleanedDicts):
  split_length= 200
  split_overlap=3
  documents = []
  for dictionary in cleanedDicts:  
    text = dictionary["text"]
    
    # split by words ensuring no sub sentence splits
    sentences = sent_tokenize(text, language="english")
    word_count = 0
    list_splits = []
    current_slice: List[str] = []
    for sen in sentences:
        current_word_count = len(sen.split(" "))
        if word_count + current_word_count > split_length:
            list_splits.append(current_slice)
            # Enable split_stride with split_by='word' while respecting sentence boundaries.
            if split_overlap:
                overlap = []
                w_count = 0
                for s in current_slice[::-1]:
                    sen_len = len(s.split(" "))
                    if w_count < split_overlap:
                        overlap.append(s)
                        w_count += sen_len
                    else:
                        break
                current_slice = list(reversed(overlap))
                word_count = w_count
            else:
                current_slice = []
                word_count = 0
        current_slice.append(sen)
        word_count += len(sen.split(" "))
    if current_slice:
        list_splits.append(current_slice)

    text_splits = []
    for sl in list_splits:
        txt = ' '.join(sl)
        if len(txt) > 0:
            text_splits.append(txt)


    # create new document dicts for each text split
    for i, txt in enumerate(text_splits):
        doc = deepcopy(dictionary)
        doc["text"] = txt
        if "meta" not in doc.keys() or doc["meta"] is None:
            doc["meta"] = {}
        doc["meta"]["_split_id"] = i
        documents.append(doc)

  return documents


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


##Creating the Web-Crawler


The Webcrawler from the official Haystack website [7] did not work in Colab because of the Google Chrome Webdriver. 

Therefore we used the provided original code from the github-website of Haystack [8] and changed the code to work with Google Colab.

We set our own chrome_options, used the webdriver from the webdriver-manager and removed the creation of textfiles, since the websites will be reloaded for real time data.

In [ ]:
import json
import logging
import re
from pathlib import Path
from haystack.preprocessor.cleaning import clean_wiki_text
from urllib.parse import urlparse
from typing import List, Any, Optional, Dict, Tuple, Union
from haystack.schema import Document, BaseComponent
logger = logging.getLogger(__name__)
!pip install webdriver-manager
!pip install selenium
!apt-get update 
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
urlsUpdate = []
alreadyUpdated = False

class Crawler(BaseComponent):
   
    outgoing_edges = 1

    def __init__(self, output_dir: str, urls: Optional[List[str]] = None, crawler_depth: int = 1,
                 filter_urls: Optional[List] = None, overwrite_existing_files=True):
        #modified chrome_options and webdriver
        chrome_options = webdriver.ChromeOptions()
        chrome_options.add_argument('--headless')
        chrome_options.add_argument('--no-sandbox')
        chrome_options.add_argument('--disable-dev-shm-usage')
        self.driver = webdriver.Chrome('chromedriver',options=chrome_options)
        self.urls = urls
        self.output_dir = output_dir
        self.crawler_depth = crawler_depth
        self.filter_urls = filter_urls
        self.overwrite_existing_files = overwrite_existing_files

    def crawl(self,  output_dir: Union[str, Path, None] = None,
              urls: Optional[List[str]] = None,
              crawler_depth: Optional[int] = None,
              filter_urls: Optional[List] = None,
              overwrite_existing_files: Optional[bool] = None) -> List[Path]:
        
        urls = urls or self.urls
        output_dir = output_dir or self.output_dir
        filter_urls = filter_urls or self.filter_urls
        file_paths: list = []
        sub_links: Dict[str, List] = {}

        # don't go beyond the initial list of urls (starting pages)
        if crawler_depth == 0:
            file_paths.extend(self._write_to_files(urls, output_dir=output_dir))
        # follow one level of sublinks (returns crawled pages)
        elif crawler_depth == 1:
            for url_ in urls:
                existed_links: List = list(sum(list(sub_links.values()), []))
                sub_links[url_] = list(self._extract_sublinks_from_url(base_url=url_, filter_urls=filter_urls,
                                                                  existed_links=existed_links))
            for url in sub_links:
                file_paths.extend(self._write_to_files(sub_links[url], output_dir=output_dir, base_url=url))

        return file_paths

#we reused the functions created by Haystack's provided github code
    def _write_to_files(self, urls: List[str], output_dir: Path, base_url: str = None) -> List[Path]:
        paths = []
        for link in urls:
            if (alreadyUpdated != True):
                urlsUpdate.append(link)
            logger.info(f"writing contents from `{link}`")
            self.driver.get(link)
            el = self.driver.find_element_by_tag_name('body')
            text = el.text

            data = {}
            
            data['meta'] = {'url': link}
            if base_url:
                data['meta']['base_url'] = base_url
            data['text'] = clean_wiki_text(text)
            paths.append(data)

        return paths

    def run(self, output_dir: Union[str, Path, None] = None, urls: Optional[List[str]] = None,  # type: ignore
            crawler_depth: Optional[int] = None, filter_urls: Optional[List] = None,  # type: ignore
            overwrite_existing_files: Optional[bool] = None, return_documents: Optional[bool] = False,  # type: ignore
            **kwargs) -> Tuple[Dict, str]:  # type: ignore
        

        file_paths = self.crawl(urls=urls, output_dir=output_dir, crawler_depth=crawler_depth,
                                  filter_urls=filter_urls, overwrite_existing_files=overwrite_existing_files)
        if return_documents:
            crawled_data = [self._read_json_file(_file) for _file in file_paths]
            results = {"documents": crawled_data}
        else:
            results = {"paths": file_paths}
        results.update(**kwargs)
        return results, "output_1"

    @staticmethod
    def _is_internal_url(base_url: str, sub_link: str) -> bool:
        base_url_ = urlparse(base_url)
        sub_link_ = urlparse(sub_link)
        return base_url_.scheme == sub_link_.scheme and base_url_.netloc == sub_link_.netloc

    @staticmethod
    def _is_inpage_navigation(base_url: str, sub_link: str) -> bool:
        base_url_ = urlparse(base_url)
        sub_link_ = urlparse(sub_link)
        return base_url_.path == sub_link_.path and base_url_.netloc == sub_link_.netloc

    def _extract_sublinks_from_url(self, base_url: str,
                                   filter_urls: Optional[List] = None,
                                  existed_links: List = None) -> set:
        self.driver.get(base_url)
        a_elements = self.driver.find_elements_by_tag_name('a')
        sub_links = set()
        if not (existed_links and base_url in existed_links):
            if filter_urls:
                if re.compile('|'.join(filter_urls)).search(base_url):
                    sub_links.add(base_url)

        for i in a_elements:
            sub_link = i.get_attribute('href')
            if not (existed_links and sub_link in existed_links):
                if self._is_internal_url(base_url=base_url, sub_link=sub_link) \
                        and (not self._is_inpage_navigation(base_url=base_url, sub_link=sub_link)):
                    if filter_urls:
                        if re.compile('|'.join(filter_urls)).search(sub_link):
                            sub_links.add(sub_link)
                    else:
                        sub_links.add(sub_link)

        return sub_links

    def _read_json_file(self, file_path: Path):
        """Read the json file and return the content"""
        with open(file_path.absolute(), "r") as read_file:
            return json.load(read_file)


     |████████████████████████████████| 904 kB 11.0 MB/s 
Get:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:5 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:7 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Hit:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ub

##Execution of the Web-Crawler


<img height="300px" src= "https://drive.google.com/uc?export=view&id=1s7lM-VxTWzzXCYnr7B9dOntNg1uAOfa0">


For the document collection of our QA-system we used the modified Webcrawler [8]. We used two different versions of the webcrawler.
The first one crawls starting pages, in this case Coinmarketcap [10] and the support of Blockchain [11] to recieve additionally all the articles linked from the starting pages.
The Wikipedia cryptocurrency website [12] included a huge collection of hyperlinks. Among those only a few articles were relevant for our QA-system.
Therefore we used a second crawler to crawl a collection of the relevant Wikipedia articles as starting pages without further crawling.

While crawling the Websites, the crawler also saved all the URLs to be able to reload the websites' data without the need to crawl the websites again.

The crawler returns the crawled texts as a dictonary.






In [ ]:
crawler = Crawler(output_dir="crawled_files")
docs = crawler.crawl(urls=["https://coinmarketcap.com/coins/", "https://support.blockchain.com/hc/en-us/categories/201149143-Cryptocurrency-FAQs"],
                     overwrite_existing_files= True,
                     crawler_depth= 1,
                     output_dir="crawled_files",
                     filter_urls= ["currencies", "support\.blockchain"])

singleDocs = crawler.crawl(urls=["https://en.wikipedia.org/wiki/Cryptocurrency",
                                 "https://en.wikipedia.org/wiki/Strong_cryptography", 
                                 "https://en.wikipedia.org/wiki/Decentralization",
                                 "https://en.wikipedia.org/wiki/Central_bank_digital_currency",
                                 "https://en.wikipedia.org/wiki/Distributed_ledger",
                                 "https://en.wikipedia.org/wiki/Blockchain",
                                 "https://en.wikipedia.org/wiki/Proof_of_work",
                                 "https://en.wikipedia.org/wiki/Proof_of_stake",
                                 "https://en.wikipedia.org/wiki/Decentralized_application",
                                 "https://en.wikipedia.org/wiki/Smart_contract",
                                 "https://en.wikipedia.org/wiki/Fungibility",
                                 "https://en.wikipedia.org/wiki/Non-fungible_token",
                                 "https://en.wikipedia.org/wiki/Satoshi_Nakamoto",
                                 "https://en.wikipedia.org/wiki/Bitcoin_network#Mining",
                                 "https://en.wikipedia.org/wiki/Peer-to-peer",
                                 "https://en.wikipedia.org/wiki/Cryptocurrency_wallet",
                                 "https://en.wikipedia.org/wiki/Public-key_cryptography",
                                 "https://en.wikipedia.org/wiki/Cryptocurrency_exchange",
                                 "https://en.wikipedia.org/wiki/Binance"],
                     overwrite_existing_files= True,
                     crawler_depth= 0,
                     output_dir="crawled_files")

alreadyUpdated = True
docs.extend(singleDocs)

##Retriever and Pipeline

This function creates the Retriever and the pipeline.
The Joined Retriever delivers the best performance for our QA-system. It uses a BM25 ElasticsearchRetriever as well as a Dense Passage Retriever and joins their results [13]. 

Haystack allows the customization of the pipeline by adding nodes [14].
First the Elasticsearch Retriever and the Dense Passage Retriever were added. Following, the JoinDocument node was added to combine the results of both retrievers. As the last step the Reader was added to the pipeline. The reader generates the answers from the retrieved documents.

In [ ]:
from haystack.pipeline import Pipeline, JoinDocuments
from haystack.retriever import DensePassageRetriever

def initRetrieverAndPipeline():

  es_retriever = ElasticsearchRetriever(document_store=document_store)
  
  dpr_retriever = DensePassageRetriever(document_store=document_store,
                                  query_embedding_model="facebook/dpr-question_encoder-single-nq-base",
                                  passage_embedding_model="facebook/dpr-ctx_encoder-single-nq-base",
                                  max_seq_len_query=64,
                                  max_seq_len_passage=256,
                                  batch_size=16,
                                  use_gpu=True,
                                  embed_title=True,
                                  use_fast_tokenizers=True)

  #Creating the embeddings for the Dense Passage Retriever 
  document_store.update_embeddings(dpr_retriever)

  p = Pipeline()
  p.add_node(component=es_retriever, name="ESRetriever", inputs=["Query"])
  p.add_node(component=dpr_retriever, name="DPRRetriever", inputs=["Query"])
  p.add_node(component=JoinDocuments(join_mode="concatenate"), name="JoinResults", inputs=["ESRetriever", "DPRRetriever"])
  p.add_node(component=reader, name="QAReader", inputs=["JoinResults"])

  return p

When the Webcrawler returns the dictionary based on the documents,
before creating the pipeline, the text from the dictionary needs to be pre-processed and saved to the document-store.
Afterwards the Retriever and the pipeline can be created, as the retrievers require the document store.

In [ ]:
global pipe

cleanedDicts =  preprocessDicts(docs)
splitDicts = splitDictionaries(cleanedDicts)
saveDataFromDictonaryToElasticseach(splitDicts)
pipe = initRetrieverAndPipeline()

09/10/2021 11:49:19 - INFO - elasticsearch -   POST http://localhost:9200/document/_delete_by_query [status:200 request:0.102s]
09/10/2021 11:49:21 - INFO - elasticsearch -   POST http://localhost:9200/document/_delete_by_query [status:200 request:0.010s]
09/10/2021 11:49:25 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:2.194s]
09/10/2021 11:49:26 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.063s]
09/10/2021 11:49:27 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.148s]
09/10/2021 11:49:28 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.077s]
09/10/2021 11:49:29 - INFO - filelock -   Lock 140554530747216 acquired on /root/.cache/huggingface/transformers/4ad08b5f983c1384baaf257d8edf51a7a3961fd8c75a1778ac604e3c0b564dd9.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99.l

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

09/10/2021 11:49:29 - INFO - filelock -   Lock 140554530747216 released on /root/.cache/huggingface/transformers/4ad08b5f983c1384baaf257d8edf51a7a3961fd8c75a1778ac604e3c0b564dd9.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99.lock
09/10/2021 11:49:30 - INFO - filelock -   Lock 140554131622544 acquired on /root/.cache/huggingface/transformers/b305bc9085b3d0ce33551c251b75c11b6c6df1d4d51e5d3439d01cf4bb1abc9d.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4.lock


Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

09/10/2021 11:49:30 - INFO - filelock -   Lock 140554131622544 released on /root/.cache/huggingface/transformers/b305bc9085b3d0ce33551c251b75c11b6c6df1d4d51e5d3439d01cf4bb1abc9d.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4.lock
09/10/2021 11:49:31 - INFO - filelock -   Lock 140554600726672 acquired on /root/.cache/huggingface/transformers/d5b5f07ee846d5baa7142e121b6ee77d11ac68bd5d4541faab38a1ea76c2954a.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79.lock


...

09/10/2021 11:49:54 - INFO - filelock -   Lock 140554137352592 released on /root/.cache/huggingface/transformers/f31ea67434695abc6c4fbe109214416d8b48a44f2fe5a0617e7faa3d6a4f8d05.be8dbf4cc0650b9c5997b3b3bc47d0d6c20749c3871e9285d3b624cd75dd9ee6.lock
09/10/2021 11:49:55 - INFO - filelock -   Lock 140554131403216 acquired on /root/.cache/huggingface/transformers/2623d56adfe8cc7bf9275b0c620a0e271ee4004c335173bde56310dc8ea99d4f.714228ba33c6248205269978fd6d0ca0ef96508cbd4a11d894882e71d45fad7c.lock


Downloading:   0%|          | 0.00/438M [00:00<?, ?B/s]

09/10/2021 11:50:09 - INFO - filelock -   Lock 140554131403216 released on /root/.cache/huggingface/transformers/2623d56adfe8cc7bf9275b0c620a0e271ee4004c335173bde56310dc8ea99d4f.714228ba33c6248205269978fd6d0ca0ef96508cbd4a11d894882e71d45fad7c.lock
09/10/2021 11:50:11 - INFO - elasticsearch -   POST http://localhost:9200/document/_count [status:200 request:0.019s]
09/10/2021 11:50:11 - INFO - haystack.document_store.elasticsearch -   Updating embeddings for all 810 docs ...


Updating embeddings:   0%|          | 0/810 [00:00<?, ? Docs/s]

Create embeddings:   0%|          | 0/816 [00:00<?, ? Docs/s]

#Website

We created our Website with the help of Anvil [15].

Open this **Website** for our QA-system Crypto-Answer:


---

https://crypto-answer-il2.anvil.app/

---





The following command connects this Google Colab notebook to the anvil Server (our Website).

In [ ]:
anvil.server.connect("O75TERSQZ3NH3S32EYMX52TH-ND325CRH3U2JZ42F")

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default environment (dev)" as SERVER


##Update data Callable

This is the method for updating the document-store content to real time data. When it is called, a new crawler is created, which crawls the documents from the links that were saved before as starting pages without further crawling.
The preprocessing step as well as the saving to the Elasticsearch document-store needs to be repeated. Following, the pipeline needs to be renewed.

In [ ]:
@anvil.server.callable
def updateWebsiteData():
    updateCrawler = Crawler(output_dir="updated_files")
    updatedDocs = crawler.crawl(urls=urlsUpdate,
                        overwrite_existing_files= True,
                        crawler_depth= 0,
                        output_dir="updated_files")
    cleanedDicts =  preprocessDicts(docs)
    splitDicts = splitDictionaries(cleanedDicts)
    saveDataFromDictonaryToElasticseach(splitDicts)
    pipe = initRetrieverAndPipeline()
    return True


#updateWebsiteData() # -> for testing in Google Colab

##Question Callable

This Callablek runs the pipeline to generate the predictions for the given question.

You can change the top_k_retriever(document amount) and the top_k_reader(answer amount) values. 
Higher values mean, that the prediction will be slower!

In [ ]:
@anvil.server.callable
def prediction(question):
  prediction = pipe.run(query=question, top_k_retriever=10, top_k_reader=4)
  return prediction 

#Alternative to using : Type in your question here: # -> for testing in Google Colab
#question = "What is the price of bitcoin?"
#print_answers(prediction(question)

#Resources

[1] J. Hosp.(2018).*Blockchain 2.0*.FinanzBuch Verlag,Munic.

[2] CoinMarketCap.(2021).*CoinMarketCap*.[Online].Available: https://coinmarketcap.com/ (zuletzt aufgerufen am 12.09.2021)

[3] Deepset-ai.(2021).*Haystack*.[Online].Available: https://github.com/deepset-ai/haystack (zuletzt aufgerufen am 12.09.2021)

[4] Hugging Face.(2021).*Hugging Face*.[Online].Available: https://huggingface.co/models (zuletzt aufgerufen am 12.09.2021)

[5] Deepset-ai.(2021).*Annotation Tool*.[Online].Available: https://haystack.deepset.ai/docs/latest/annotationmd (zuletzt aufgerufen am 12.09.2021)

[6] Deepset-ai.(2021).*DocumentStores*.[Online].Available: https://haystack.deepset.ai/usage/document-store (zuletzt aufgerufen am 12.09.2021)

[7] Deepset-ai.(2021).*Module crawler*.[Online].Available: https://haystack.deepset.ai/reference/crawler (zuletzt aufgerufen am 12.09.2021)

[8] deepset-ai.(2021).*crawler.py*.[Online].Available: https://github.com/deepset-ai/haystack/blob/master/haystack/connector/crawler.py (zuletzt aufgerufen am 11.09.2021)

[9] deepset-ai.(2021).*preprocessor.py*.[Online].Available: https://github.com/deepset-ai/haystack/blob/master/haystack/preprocessor/preprocessor.py (zuletzt aufgerufen 12.09.2021)

[10] Coinmarketcap.(2021).*Top 100 Crypto Coins by Market capitalization*.[Online].Available: https://coinmarketcap.com/coins/ (zuletzt aufgerufen am 16.09.2021)

[11] Blockchain.(2020).*Cryptocurrency FAQs*.[Online].Availabe: https://support.blockchain.com/hc/en-us/categories/201149143-Cryptocurrency-FAQs (zuletzt aufgerufen am 12.09.2021)

[12] Wikipedia.(2021).*Cryptocurrency*.[Online].Available: https://en.wikipedia.org/wiki/Cryptocurrency (zuletzt aufgerufen am 12.9.2021)

[13] Deepset-ai.(2021).*Pipelines Tutorial*.[Online].Availabe: https://haystack.deepset.ai/tutorials/pipelines (zuletzt aufgerufen am 14.9.2021)

[14] Deepset-ai.(2021).*Module Pipeline*.[Online].Availabe: https://haystack.deepset.ai/reference/pipelines (zuletzt aufgerufen am 15.9.2021)

[15] Anvil.(2020).*Tutorials*.[Online].Available: https://anvil.works/learn/tutorials (zuletzt aufgerufen am 12.9.2021)



